In [11]:
import sys
from pathlib import Path
current_dir = Path().resolve()
sys.path.append(current_dir.parent.parent.as_posix())

import os
from os import listdir
from os.path import isfile, join

# %% General setup
import pandas as pd
from pathlib import Path
from data_io import DataIO

import utils
import numpy as np
from utils import make_figure, save_fig
from scipy.stats import wilcoxon
from audrey.preprocessing.project_colors import ProjectColors

# Load data
session_id = '251014_A'
ssd_dir = Path('r/media/aleong/Audrey-experiments')
data_dir = Path(r'/media/aleong/Audrey-experiments/dataset')
figure_dir = data_dir / Path(session_id)

if not os.path.exists(figure_dir):
    os.makedirs(figure_dir)
    print(f"Created the figure folder : {figure_dir}")
else:
    print(f"Figure folder already exists.")

data_io = DataIO(data_dir)
loadname = data_dir / f'{session_id}_cells.csv'
data_io.load_session(session_id, load_pickle=False, load_waveforms=False)
data_io.dump_as_pickle()

cells_df = pd.read_csv(loadname, header=[0, 1], index_col=0)
clrs = ProjectColors()

# INCLUDE_RANGE = 50  # include cells at max distance = 50 um

Figure folder already exists.


### Select the recordings to analyse

In [16]:
raw_path = Path('/media/aleong/Audrey-experiments/251014_A/sorted')
recording_names = [Path(f).stem for f in listdir(raw_path) if isfile(join(raw_path, f)) and f.endswith('.raw')]

print('Which of the following recording are the pa-prr and light-pa-prr recordings : ')
for num, rec in enumerate(recording_names):
    print(f'\t{num} --> {rec}')
pa_prr_rec_nr = int(input('PA PRR recording : '))
light_pa_prr_rec_nr = int(input('Light PA PRR recording : '))

selected_rec_names = [
    recording_names[pa_prr_rec_nr],
    recording_names[light_pa_prr_rec_nr],
]

print(f'Selected recordings: {selected_rec_names[0]}\n \t\t     {selected_rec_names[1]}')

Which of the following recording are the pa-prr and light-pa-prr recordings : 
	0 --> 251014_A_001_noblocker_light_checkerboard_30sq20px
	1 --> 251014_A_002_noblocker_light_checkerboard_12sq50px
	2 --> 251014_A_003_noblocker_light_chirp
	3 --> 251014_A_004_noblocker_light_SWN_30Hz
	4 --> 251014_A_005_noblocker_pa_prr_series
	5 --> 251014_A_006_noblocker_light_intensity_series
	6 --> 251014_A_007_noblocker_light_pa_prr_series
	7 --> 251014_A_008_noblocker_light_series


PA PRR recording :  4
Light PA PRR recording :  6


Selected recordings: 251014_A_005_noblocker_pa_prr_series
 		     251014_A_007_noblocker_light_pa_prr_series


### Detect the electrode stimulation site with the most significant responses, per cell

In [17]:
# %% Detect electrode stim site with most significant responses, per cell


electrodes = [[ec] for ec in data_io.burst_df.electrode.unique()]
#electrodes = [[158], [129], [113], [171], [54], [57], [38]] # if electrodes are changed in the middle of a session

pref_ec_dict = {}

for cluster_id in data_io.cluster_df.index.values:
    pref_ec = None
    n_sig_pref_ec = None

    max_fr = None
    for ec in electrodes:
        if pd.isna(ec):
            continue
            
        df = data_io.burst_df.query(f'electrode in {ec}')
        tids = df.train_id.unique()
        n_sig = 0
        for tid in tids:
            if cells_df.loc[cluster_id, (tid, 'is_significant')] is True:
                n_sig += 1

        if n_sig > 1:
            if pref_ec is None or n_sig > n_sig_pref_ec:
                pref_ec = ec
                n_sig_pref_ec = n_sig

    pref_ec_dict[cluster_id] = pref_ec


print('Electrode stimulation site with the most significant responses per cell detected.\n\n------ End Of Cell ------')

Electrode stimulation site with the most significant responses per cell detected.

------ End Of Cell ------


### Find the electrode number for the light prr series

In [18]:
# Patch electrode nr for light-prr series
# electrodes = [[ec] for ec in data_io.burst_df.electrode.unique()]
burst_i = 0
current_ec_i = 0
for i, r in data_io.burst_df.query('recording_name == @selected_rec_names[1]').iterrows():

    print(i, burst_i, current_ec_i)
    data_io.burst_df.at[i, 'electrode'] = electrodes[current_ec_i][0]

    burst_i += 1
    if burst_i == 20:
        burst_i = 0
        current_ec_i += 1

        if current_ec_i == len(electrodes):
            current_ec_i = 0

print('\n------ End Of Cell ------')

251014_A_007_noblocker_light_pa_prr_series-0 0 0
251014_A_007_noblocker_light_pa_prr_series-1 1 0
251014_A_007_noblocker_light_pa_prr_series-2 2 0
251014_A_007_noblocker_light_pa_prr_series-3 3 0
251014_A_007_noblocker_light_pa_prr_series-4 4 0
251014_A_007_noblocker_light_pa_prr_series-5 5 0
251014_A_007_noblocker_light_pa_prr_series-6 6 0
251014_A_007_noblocker_light_pa_prr_series-7 7 0
251014_A_007_noblocker_light_pa_prr_series-8 8 0
251014_A_007_noblocker_light_pa_prr_series-9 9 0
251014_A_007_noblocker_light_pa_prr_series-10 10 0
251014_A_007_noblocker_light_pa_prr_series-11 11 0
251014_A_007_noblocker_light_pa_prr_series-12 12 0
251014_A_007_noblocker_light_pa_prr_series-13 13 0
251014_A_007_noblocker_light_pa_prr_series-14 14 0
251014_A_007_noblocker_light_pa_prr_series-15 15 0
251014_A_007_noblocker_light_pa_prr_series-16 16 0
251014_A_007_noblocker_light_pa_prr_series-17 17 0
251014_A_007_noblocker_light_pa_prr_series-18 18 0
251014_A_007_noblocker_light_pa_prr_series-19 19 0


In [19]:
print(data_io.burst_df.query('recording_name == @selected_rec_names[0]').shape)
print(data_io.burst_df.query('recording_name == @selected_rec_names[1]').shape)

data_io.burst_df.columns
for i, r in data_io.burst_df.query('recording_name == @selected_rec_names[1]').iterrows():
    print(i, r.electrode)

print('\n------ End Of Cell ------')

(600, 54)
(480, 54)
251014_A_007_noblocker_light_pa_prr_series-0 17.0
251014_A_007_noblocker_light_pa_prr_series-1 17.0
251014_A_007_noblocker_light_pa_prr_series-2 17.0
251014_A_007_noblocker_light_pa_prr_series-3 17.0
251014_A_007_noblocker_light_pa_prr_series-4 17.0
251014_A_007_noblocker_light_pa_prr_series-5 17.0
251014_A_007_noblocker_light_pa_prr_series-6 17.0
251014_A_007_noblocker_light_pa_prr_series-7 17.0
251014_A_007_noblocker_light_pa_prr_series-8 17.0
251014_A_007_noblocker_light_pa_prr_series-9 17.0
251014_A_007_noblocker_light_pa_prr_series-10 17.0
251014_A_007_noblocker_light_pa_prr_series-11 17.0
251014_A_007_noblocker_light_pa_prr_series-12 17.0
251014_A_007_noblocker_light_pa_prr_series-13 17.0
251014_A_007_noblocker_light_pa_prr_series-14 17.0
251014_A_007_noblocker_light_pa_prr_series-15 17.0
251014_A_007_noblocker_light_pa_prr_series-16 17.0
251014_A_007_noblocker_light_pa_prr_series-17 17.0
251014_A_007_noblocker_light_pa_prr_series-18 17.0
251014_A_007_noblocke

In [26]:
cluster_ids = data_io.cluster_df.index.values
cluster_id = cluster_ids[2]

rec_name = selected_rec_names[0]
ec = pref_ec_dict[cluster_id]
d_select = data_io.burst_df.query('electrode == @ec and '
                                              'recording_name == @rec_name').copy()
d_select.sort_values('duty_cycle', inplace=True)
        # repetition_frequencies = d_select.repetition_frequency.unique()

#duty_cycles = d_select.laser_duty_cycle.unique()
duty_cycles = d_select.duty_cycle.unique()
dc = duty_cycles[0]


tid   = d_select.query('laser_duty_cycle == @dc').iloc[0].train_id
cluster_data = utils.load_obj(data_dir / 'bootstrapped' / f'bootstrap_{cluster_id}.pkl')
print(cluster_data[tid])

{'bins': array([-200, -195, -190, -185, -180, -175, -170, -165, -160, -155, -150,
       -145, -140, -135, -130, -125, -120, -115, -110, -105, -100,  -95,
        -90,  -85,  -80,  -75,  -70,  -65,  -60,  -55,  -50,  -45,  -40,
        -35,  -30,  -25,  -20,  -15,  -10,   -5,    0,    5,   10,   15,
         20,   25,   30,   35,   40,   45,   50,   55,   60,   65,   70,
         75,   80,   85,   90,   95,  100,  105,  110,  115,  120,  125,
        130,  135,  140,  145,  150,  155,  160,  165,  170,  175,  180,
        185,  190,  195,  200,  205,  210,  215,  220,  225,  230,  235,
        240,  245,  250,  255,  260,  265,  270,  275,  280,  285,  290,
        295,  300,  305,  310,  315,  320,  325,  330,  335,  340,  345,
        350,  355,  360,  365,  370,  375,  380,  385,  390,  395,  400,
        405,  410,  415,  420,  425,  430,  435,  440,  445,  450,  455,
        460,  465,  470,  475,  480,  485,  490,  495]), 'bin_size': 20, 'binned_sp': array([[0, 0, 0, ..., 0, 0, 0

### Plot the raster plots for each cell

In [28]:
##%% Plot raster plots for each individual cell

cluster_ids = data_io.cluster_df.index.values

electrodes  = data_io.burst_df.electrode.unique()

for cluster_id in cluster_ids:
    cluster_data = utils.load_obj(data_dir / 'bootstrapped' / f'bootstrap_{cluster_id}.pkl')

    n_electrodes = electrodes.size

    ec = pref_ec_dict[cluster_id]
    if ec is None:    # Skip clusters without preferred electrode
        print(f"None, cluster {cluster_id}")
        continue

    # Setup figure layout
    fig = utils.make_figure(
        width    =1,
        height   =1.5,
        x_domains={
            1: [[0.1, 0.9]],
        },
        y_domains={
            1: [[0.1, 0.9]]
        },
    )

    # Setup variables for plotting
    burst_offset   = 0
    x_plot, y_plot = [], []
    yticks         = []
    ytext          = []
    pos            = dict(row=1, col=1)

    has_sig = False

    for rec_i, rec_name in enumerate(selected_rec_names):
        d_select = data_io.burst_df.query('electrode == @ec and '
                                              'recording_name == @rec_name').copy()
        d_select.sort_values('duty_cycle', inplace=True)
        # repetition_frequencies = d_select.repetition_frequency.unique()

        if '_light_pa_' in rec_name:
        #if 'light-prr' in rec_name:
            duty_cycles = d_select.laser_duty_cycle.unique()
        elif '_pa_' in rec_name:
        #elif 'prr' in rec_name:
            duty_cycles = d_select.duty_cycle.unique()

        for dc in duty_cycles:
            if '_light_pa_' in rec_name:
            #if 'light-prr' in rec_name:
                tid   = d_select.query('laser_duty_cycle == @dc').iloc[0].train_id
                frep  = data_io.burst_df.query('train_id == @tid').iloc[0].laser_duty_cycle
                bd    = data_io.burst_df.query('train_id == @tid').iloc[0].laser_burst_duration
                rname = 'PA+light'
            elif '_pa_' in rec_name:
            #elif 'prr' in rec_name:
                tid   = d_select.query('duty_cycle == @dc').iloc[0].train_id
                frep  = data_io.burst_df.query('train_id == @tid').iloc[0].duty_cycle
                bd    = data_io.burst_df.query('train_id == @tid').iloc[0].burst_duration
                rname = 'PA'

        
            spike_times = cluster_data[tid]['spike_times']
            bins = cluster_data[tid]['bins']

            ytext.append(f'dc: {frep:.0f} bd: {bd:.0f}, {rec_i}, {rname}')
            yticks.append(burst_offset + len(spike_times) / 2)

            for burst_i, sp in enumerate(spike_times):
                x_plot.append(np.vstack([sp, sp, np.full(sp.size, np.nan)]).T.flatten())
                y_plot.append(np.vstack([np.ones(sp.size) * burst_offset,
                                         np.ones(sp.size)* burst_offset +1, np.full(sp.size, np.nan)]).T.flatten())
                burst_offset += 1

    x_plot = np.hstack(x_plot)
    y_plot = np.hstack(y_plot)

    fig.add_scatter(
        x = x_plot, y = y_plot,
        mode = 'lines', line = dict(color='black', width=0.5),
        showlegend = False,
        **pos,
    )

    fig.update_xaxes(
        tickvals = np.arange(-500, 500, 100),
        title_text = f'time [ms]',
        range = [bins[0]-1, bins[-1]+1],
        **pos,
    )

    fig.update_yaxes(
        #range=[0, n_bursts],
        tickvals = yticks,
        ticktext = ytext,
        **pos,
    )

    sname = figure_dir  / 'raster plots' / f'{cluster_id}'

    utils.save_fig(fig, sname, display=False)

print('\n\t\t\t------ End Of Cell ------')

None, cluster uid_251014_A_000
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_001.png
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_002.png
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_003.png
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_004.png
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_005.png
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_006.png
None, cluster uid_251014_A_007
None, cluster uid_251014_A_008
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_009.png
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_010.png
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_011.png
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_012.png
saved: /media/aleong/Audrey/dataset/251014_A/raster plots/uid_251014_A_013.png
saved: /media/aleong/Audrey/dataset/25

In [24]:
d_select.laser_duty_cycle

251014_A_007_noblocker_light_pa_prr_series-40      8.0
251014_A_007_noblocker_light_pa_prr_series-41      8.0
251014_A_007_noblocker_light_pa_prr_series-42      8.0
251014_A_007_noblocker_light_pa_prr_series-43      8.0
251014_A_007_noblocker_light_pa_prr_series-44      8.0
                                                  ... 
251014_A_007_noblocker_light_pa_prr_series-475    29.0
251014_A_007_noblocker_light_pa_prr_series-476    29.0
251014_A_007_noblocker_light_pa_prr_series-477    29.0
251014_A_007_noblocker_light_pa_prr_series-478    29.0
251014_A_007_noblocker_light_pa_prr_series-479    29.0
Name: laser_duty_cycle, Length: 80, dtype: float32

In [14]:
'''
cluster_ids = data_io.cluster_df.index.values
blockers = ['noblocker', 'cpp', 'washout']

for cluster_id in cluster_ids:

    ec = pref_ec_dict[cluster_id]
    if ec is None:
        continue
    blocker = "noblocker"
    d_select = data_io.burst_df.query('electrode == @ec and '
                                              'Blocker == @blocker')
    d_select.sort_values('duty_cycle', inplace=True)
    duty_cycles = d_select.duty_cycle.unique()
    fmax = np.max(duty_cycles)
    print(d_select.query('duty_cycle == @fmax'))
'''

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

Empty DataFrame
Columns: [rec_id, burst_id, dmd_train_onset, dmd_burst_onset, dmd_burst_offset, train_id, Recording Number, Retina Slice, Blocker, Blocker Admission, Washout Start, Animal, Animal Birthdate, Implant, Implant Diameter, Laser, Connected Fibers, Mea, Medium, Attenuators, Laser level, Inter protocol interval, inter_trial_interval, _protocol_name, baseline_duration, laser_burst_duration, laser_burst_period, laser_burst_count, dmd_baseline_duration, dmd_spot_size_um, dmd_frame_freq, stim_offset, dmd_burst_duration, dmd_burst_period, dmd_burst_count, laser_duty_cycle, laser_prr, rec_file, burst_period, burst_count, burst_duration, duty_cycle, electrode, protocol, pos_xyz, recording_name, rec_train_i, laser_x, laser_y, stimtype, laser_train_onset, laser_burst_onset, laser_burst_offset]
Index: []

[0 rows x 53 columns]
Empty DataFrame
Columns: [rec_id, burst_id, dmd_train_onset, dmd_burst_onset, dmd_burst_offset, train_id, Recording Number, Retina Slice, Blocker, Blocker Admissi

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

Empty DataFrame
Columns: [rec_id, burst_id, dmd_train_onset, dmd_burst_onset, dmd_burst_offset, train_id, Recording Number, Retina Slice, Blocker, Blocker Admission, Washout Start, Animal, Animal Birthdate, Implant, Implant Diameter, Laser, Connected Fibers, Mea, Medium, Attenuators, Laser level, Inter protocol interval, inter_trial_interval, _protocol_name, baseline_duration, laser_burst_duration, laser_burst_period, laser_burst_count, dmd_baseline_duration, dmd_spot_size_um, dmd_frame_freq, stim_offset, dmd_burst_duration, dmd_burst_period, dmd_burst_count, laser_duty_cycle, laser_prr, rec_file, burst_period, burst_count, burst_duration, duty_cycle, electrode, protocol, pos_xyz, recording_name, rec_train_i, laser_x, laser_y, stimtype, laser_train_onset, laser_burst_onset, laser_burst_offset]
Index: []

[0 rows x 53 columns]
Empty DataFrame
Columns: [rec_id, burst_id, dmd_train_onset, dmd_burst_onset, dmd_burst_offset, train_id, Recording Number, Retina Slice, Blocker, Blocker Admissi

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_493179/3887833623.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

In [16]:
##%% optimized single-panel firing rate plotting
DEBUG = True   # set to False to mute debug output

def dbg(*msg):
    if DEBUG:
        print("[DEBUG]", *msg)

cluster_ids = data_io.cluster_df.index.values

selected_rec_names = [
    pa_prr_rec_name,
    light_pa_prr_rec_name,
]

plot_titles = [
    'PA',
    'PA + light',
]

dbg("Starting plotting for", len(cluster_ids), "clusters")

for cluster_id in cluster_ids:

    dbg(f"\n=== Processing cluster {cluster_id} ===")

    # Create a *single panel* figure
    fig = utils.make_figure(
        width    = 1,
        height   = 1.5,
        x_domains={1: [[0.1, 0.9]]},
        y_domains={1: [[0.1, 0.9]]},
    )

    # Always plot into (row=1, col=1)
    pos = dict(row=1, col=1)

    # Load data for this cluster
    dbg("Loading bootstrap data…")
    cluster_data = utils.load_obj(
        data_dir / 'bootstrapped' / f'bootstrap_{cluster_id}.pkl'
    )

    # preferred electrode
    ec = pref_ec_dict[cluster_id]
    dbg("Preferred electrode:", ec)

    if ec is None:
        dbg("Skipping cluster (no preferred electrode)")
        continue

    # Track global y-limits
    y_max = 0

    # Loop over all recordings (all curves go on same axis)
    for rec_i, rec_name in enumerate(selected_rec_names):

        dbg(f"  Recording: {rec_name}")

        # Filter data
        d_select = data_io.burst_df.query(
            'electrode == @ec and recording_name == @rec_name'
        ).copy()

        dbg("    Burst trials found:", len(d_select))

        if len(d_select) == 0:
            dbg("    WARNING: No trials match this electrode/recording")
            continue

        d_select.sort_values('duty_cycle', inplace=True)

        # Determine condition type: PA vs PA+light
        if '_light_pa_' in rec_name:
            duty_cycles = d_select.laser_duty_cycle.unique()
            dbg("    Condition detected: PA + light")
            mode = 'light'
        elif '_pa_' in rec_name:
            duty_cycles = d_select.duty_cycle.unique()
            dbg("    Condition detected: PA")
            mode = 'pa'
        else:
            dbg("    WARNING: Unrecognized condition; skipping")
            continue

        dbg("    Duty cycles:", duty_cycles)

        for dc in duty_cycles:
            dbg(f"      Duty cycle: {dc}")

            # Extract train ID + burst settings
            if mode == 'light':
                row0 = d_select.query('laser_duty_cycle == @dc').iloc[0]
                tid  = row0.train_id

                bdf  = data_io.burst_df
                frep = bdf.query('train_id == @tid').iloc[0].laser_duty_cycle
                bd   = bdf.query('train_id == @tid').iloc[0].laser_burst_duration

                clr_a = clrs.padmd_stim(dc, alpha = 0.2)
                clr   = clrs.padmd_stim(dc)

            elif mode == 'pa':
                row0 = d_select.query('duty_cycle == @dc').iloc[0]
                tid  = row0.train_id

                bdf  = data_io.burst_df
                frep = bdf.query('train_id == @tid').iloc[0].duty_cycle
                bd   = bdf.query('train_id == @tid').iloc[0].burst_duration

                clr_a = clrs.duty_cycle(dc, alpha = 0.2)
                clr   = clrs.duty_cycle(dc)

            dbg("        train_id:", tid, "| burst dur:", bd)

            # Load bootstrap FR data
            bins    = cluster_data[tid]['bins']
            sp      = cluster_data[tid]['firing_rate']
            ci_low  = cluster_data[tid]['firing_rate_ci_low']
            ci_high = cluster_data[tid]['firing_rate_ci_high']

            if sp is None:
                dbg("        WARNING: No spike data for tid", tid)
                continue

            y_max = max(y_max, ci_high.max())

            # --- shaded burst window ---
            fig.add_scatter(
                x=[0, 0, bd, bd],
                y=[0, 1000, 1000, 0],
                mode='lines',
                line=dict(width=0),
                fill='toself',
                fillcolor='rgba(128, 128, 128, 0.1)',
                showlegend=False,
                **pos,
            )

            # --- confidence interval ---
            fig.add_scatter(
                x=bins,
                y=ci_low,
                line=dict(width=0),
                showlegend=False,
                **pos,
            )
            fig.add_scatter(
                x=bins,
                y=ci_high,
                mode='lines',
                line=dict(width=0),
                fill='tonexty',
                fillcolor=clr_a,
                showlegend=False,
                **pos,
            )

            # --- firing-rate line ---
            legend_name = f"DC {dc}"
            show_leg = not any(trace.name == legend_name for trace in fig.data)

            fig.add_scatter(
                x=bins,
                y=sp,
                name=legend_name,
                line=dict(width=1, color=clr),
                showlegend=show_leg,
                **pos,
            )

    # Final axes formatting
    fig.update_xaxes(
        tickvals=np.arange(-500, 500, 100),
        title_text='Time (ms)',
        range=[-101, 351],
        **pos,
    )
    fig.update_yaxes(
        title_text='Firing rate (Hz)',
        tickvals=np.arange(0, np.ceil(y_max/10)*10 + 1, 10),  # nice 10-Hz spacing
        range=[0, y_max],
        **pos,
    )

    # Save figure
    sname = figure_dir / 'firing_rate_per_condition' / f'{cluster_id}'
    dbg("Saving figure to:", sname)

    utils.save_fig(fig, sname, display=False, formats=['png', 'svg'])

dbg("\nAll clusters processed.")


[DEBUG] Starting plotting for 85 clusters
[DEBUG] 
=== Processing cluster uid_251014_A_000 ===
[DEBUG] Loading bootstrap data…
[DEBUG] Preferred electrode: None
[DEBUG] Skipping cluster (no preferred electrode)
[DEBUG] 
=== Processing cluster uid_251014_A_001 ===
[DEBUG] Loading bootstrap data…
[DEBUG] Preferred electrode: [194.0]
[DEBUG]   Recording: 251014_A_005_noblocker_pa_prr_series
[DEBUG]     Burst trials found: 100
[DEBUG]     Condition detected: PA
[DEBUG]     Duty cycles: [ 8. 12. 20. 29. 37.]
[DEBUG]       Duty cycle: 8.0
[DEBUG]         train_id: tid_251014_A_015 | burst dur: 10.0
[DEBUG]       Duty cycle: 12.0
[DEBUG]         train_id: tid_251014_A_016 | burst dur: 10.0
[DEBUG]       Duty cycle: 20.0
[DEBUG]         train_id: tid_251014_A_017 | burst dur: 10.0
[DEBUG]       Duty cycle: 29.0
[DEBUG]         train_id: tid_251014_A_018 | burst dur: 10.0
[DEBUG]       Duty cycle: 37.0
[DEBUG]         train_id: tid_251014_A_019 | burst dur: 10.0
[DEBUG]   Recording: 251014_A_00

In [58]:
##%% plot individual firing rates
cluster_ids = data_io.cluster_df.index.values

blockers = ['noblocker', 'cpp', 'washout']

for cluster_id in cluster_ids:
    # Setup figure layout
    fig = utils.make_figure(
        width    = 1,
        height   = 1.5,
        x_domains= {
            1: [[0.1, 0.9]],
        },
        y_domains= {
            1: [[0.1, 0.9]]
        },
    )

    # Setup variables for plotting
    burst_offset   = 0
    x_plot, y_plot = [], []
    yticks         = []
    ytext          = []
    pos            = dict(row=1, col=1)

    has_sig = False
    
    # Load cluster data
    cluster_data = utils.load_obj(data_dir / 'bootstrapped' / f'bootstrap_{cluster_id}.pkl')

    # Get preferred electrode for this cluster
    ec = pref_ec_dict[cluster_id]
    if ec is None:
        continue


    for blocker in blockers:
        d_select = data_io.burst_df.query('electrode == @ec and '
                                              'Blocker == @blocker').copy()
        d_select.sort_values('duty_cycle', inplace=True)
        duty_cycles = d_select.duty_cycle.unique()
        
        if np.size(duty_cycles) == 0:
            continue
        fmax = np.max(duty_cycles)
        if np.isnan(fmax) :
            continue

        tid = d_select.query('duty_cycle == @fmax').iloc[0].train_id

        dc = data_io.burst_df.query('train_id == @tid').iloc[0].duty_cycle
        bins = cluster_data[tid]['bins']
        fr = cluster_data[tid]['firing_rate']
        fr_ci_low = cluster_data[tid]['firing_rate_ci_low']
        fr_ci_high = cluster_data[tid]['firing_rate_ci_high']

        if 'noblocker' in blocker:
            clr = clrs.blocker_color('none', 1)
            clr_a = clrs.blocker_color('none', 0.1)

        elif 'lap4' in blocker and 'acet' not in blocker:
            clr = clrs.blocker_color('lap4', 1)
            clr_a = clrs.blocker_color('lap4', 0.1)

        # Plot single cell response curves
        # Plot the strongest stim condition for 1 PA, 1 DMD and 1 PADMD session

        elif 'washout' in blocker:
            clr = clrs.blocker_color('washout', 1)
            clr_a = clrs.blocker_color('washout', 0.1)

        else:
            clr = None
            clr_a = None

        fig.add_scatter(
            x=bins, y=fr_ci_low,
            mode='lines', line=dict(color=clr_a, width=0),
            showlegend=False,
            name=blocker,
            **pos,
        )
        fig.add_scatter(
            x=bins, y=fr_ci_high,
            mode='lines', line=dict(color=clr_a, width=0),
            showlegend=False,
            name=blocker,
            fill='tonexty',
            **pos,
        )

        fig.add_scatter(
            x=bins, y=fr,
            mode='lines', line=dict(color=clr, width=0.5),
            showlegend=True,
            name=blocker,
            **pos,
        )

    fig.update_xaxes(
        tickvals=np.arange(-500, 500, 100),
        title_text=f'time [ms]',
        range=[bins[0]-1, bins[-1]+1],
        **pos,
    )

    fig.update_yaxes(
        # range=[0, n_bursts],
        tickvals=np.arange(0, 300, 30),
        title_text=f'firing rate [Hz]',
        **pos,
    )

    sname = figure_dir / session_id / 'firing rate plots' / f'{cluster_id}'

    utils.save_fig(fig, sname, display=False)


print('\n\t\t\t------ End Of Cell ------')

saved: /media/aleong/Audrey/dataset/251014_A/251014_A/firing rate plots/uid_251014_A_001.png


KeyboardInterrupt: 

In [ ]:



##%% Gather data for Figure CPP CNQX Paper (used in the axorus-analysis libray
##%% Gather data for final plot

blockers = ['noblocker', 'cpp', 'washout']
df_save = pd.DataFrame()

for cluster_id, electrode in pref_ec_dict.items():
    if electrode is None:
        continue

    cluster_x = data_io.cluster_df.loc[cluster_id, 'cluster_x']
    df_save.at[cluster_id, f'x_mea'] = cluster_x
    cluster_y = data_io.cluster_df.loc[cluster_id, 'cluster_y']
    df_save.at[cluster_id, f'y_mea'] = cluster_y

    for blocker in blockers:
        d_select = data_io.burst_df.query('electrode in @electrode and '
                                              'blockers == @blocker').copy()
        tid = d_select.loc[d_select['duty_cycle'].idxmax()].train_id

        laser_x = data_io.burst_df.query('train_id == @tid').laser_x.values[0]
        laser_y = data_io.burst_df.query('train_id == @tid').laser_y.values[0]
        d = np.sqrt((laser_x - cluster_x) ** 2 + (laser_y - cluster_y) ** 2)

        df_save.at[cluster_id, f'{blocker} baseline'] = cells_df.loc[cluster_id, tid].baseline_firing_rate
        df_save.at[cluster_id, f'{blocker} response'] = cells_df.loc[cluster_id, tid].response_firing_rate
        df_save.at[cluster_id, f'laser_distance'] = d
        df_save.at[cluster_id, f'{blocker} is_sig'] = cells_df.loc[cluster_id, tid].is_significant
        df_save.at[cluster_id, f'{blocker} response_latency'] = cells_df.loc[cluster_id, tid].response_latency
savename = figure_dir / 'stats_data_250606_A.csv'
df_save.to_csv(savename)
print(f'Saved data in: {savename}')

# Load analysis results
df_out = pd.DataFrame()

for blocker in blockers:


    laser_x, laser_y = get_electrode_pos(stimsites[sid][tid])

    for cluster_id in data_io.unit_df.index.values:

        cluster_x = data_io.unit_df.loc[cluster_id, 'x_mea']
        cluster_y = data_io.unit_df.loc[cluster_id, 'y_mea']
        d = np.sqrt((laser_x - cluster_x) ** 2 + (laser_y - cluster_y) ** 2)
        df_out.at[cluster_id, f'd'] = d

        if response_stats[cluster_id].loc[tid, 'is_sig'] is True:
            df_out.at[cluster_id, f'{blocker}'] = True
        else:
            df_out.at[cluster_id, f'{blocker}'] = False

    data_out[f'{sid}'] = df_out

##%% Gather data for final plot

blockers = ['noblocker', 'cpp', 'washout']
df_plot = pd.DataFrame()

for cluster_id, electrode in pref_ec_dict.items():
    if electrode is None:
        continue

    for blocker in blockers:
        d_select = data_io.burst_df.query('electrode in @electrode and '
                                              'blockers == @blocker').copy()
        tid = d_select.loc[d_select['duty_cycle'].idxmax()].train_id
        print(tid)
        df_plot.at[cluster_id, f'{blocker} baseline'] = cells_df.loc[cluster_id, tid].baseline_firing_rate
        df_plot.at[cluster_id, f'{blocker} response'] = cells_df.loc[cluster_id, tid].response_firing_rate


def print_wilcoxon(d0, d1, tag1, tag2):

    idx = np.where(pd.notna(d0) & pd.notna(d1))[0]

    r, p = wilcoxon(d0[idx], d1[idx])
    d0_m = np.mean(d0[idx])
    d0_s = np.std(d0[idx])
    d1_m = np.mean(d1[idx])
    d1_s = np.std(d1[idx])
    print(f'{tag1} vs {tag2}:({d0_m:.0f} ({d0_s:.0f}), {d1_m:.0f} ({d1_s:.0f})) T = {r:.0f} (p={p:.3f})')

# ## FIGURE SETUP
n_rows = 1
n_cols = 1

y_top = 0.1
y_bottom = 0.1
yspacing = 0.1  # y space between rows

xoffset = 0.1  # x space left and right of plots
xspacing = 0.1  # x space between columns

yheight = (1 - y_bottom - y_top - yspacing * (n_rows - 1))  # height of each plot
rel_heights = [1, 1, 1]

# Generate x and y spacing for all the subplots
y_domains = dict()
y1 = 1 - y_top
for i in range(n_rows):
    row_h = yheight * rel_heights[i]
    y0 = y1 - row_h
    y_domains[i + 1] = [[y0, y1] for j in range(n_cols)]

    y1 -= (row_h + yspacing)

xwidth = (1 - (n_cols - 1) * xoffset - xspacing - 0.05) / n_cols
sx = [[xoffset + (xspacing + xwidth) * i, xoffset + (xspacing + xwidth) * i + xwidth] for i in range(n_cols)]
clrs = ProjectColors()

# Generate the figure
fig = make_figure(
    width=0.3, height=0.6,
    x_domains={
        1: sx,
    },
    y_domains=y_domains,
    subplot_titles={
        1: ['', '', ],
    },
)

xpos = [0, 1, 3, 4, 6, 7]
xdata = ['noblocker baseline', 'noblocker response',
         'cpp baseline', 'cpp response',
         'washout baseline', 'washout response']

xlbl = ['no blocker', 'lap4', 'lap4+acet', 'washout']

n_pts = df_plot.shape[0]


box_specs = dict(
    name='P23H',
    boxpoints='all',
    marker=dict(color=clrs.animal_color('P23H', 1, 1), size=2),
    line=dict(color=clrs.animal_color('P23H', 1, 1), width=1.5),
    showlegend=False,
)

for xp, xd in zip(xpos, xdata):
    fig.add_box(
        x=np.ones(n_pts) * xp,
        y=df_plot[xd].values,
        **box_specs,
    )

fig.update_yaxes(
    range=[0, 250],
    title_text=f'fr. [Hz]',
    tickvals=np.arange(0, 300, 50),
)
fig.update_xaxes(
    tickvals=[0.5, 3.5, 6.5],
    ticktext=['no blocker', 'cpp', 'washout'],
)

sname = figure_dir / f'{session_id}_boxplot'
save_fig(fig, sname, formats=['png', 'svg'], scale=3)
